# OpenAIChatCompletionClient Tutorial

## Learning Objectives

By the end of this tutorial, you will understand:
- How to make direct API calls to OpenAI using AutoGen's `OpenAIChatCompletionClient`
- When to use direct API calls vs. AutoGen's AgentChat framework
- Best practices for error handling

## Introduction

The `OpenAIChatCompletionClient` [[api docs](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.models.openai.html#autogen_ext.models.openai.OpenAIChatCompletionClient)] provides a direct interface to OpenAI's Chat Completion API. This is useful when you need:

- **Simple, synchronous conversations** with a single AI model
- **Direct control** over API parameters and response handling
- **Integration** into existing applications without the full agent framework
- **Testing and prototyping** before building more complex multi-agent systems

For more sophisticated scenarios involving multiple agents, tool usage, or complex conversation flows, you'll want to use AutoGen's AgentChat framework (covered in the following tutorials).

## Environment Setup

### API Key Configuration

Before starting, ensure you have an OpenAI API key. There are several ways to configure this:

1. **Environment variable (recommended)**: We used a `.env` file with `OPENAI_API_KEY=your_key_here`
2. **Direct parameter**: Pass the API key directly to the client (not recommended for production)
3. **System environment**: Export the key in your shell session

Let's verify the API key is available as an environment variable:

In [1]:
import os

api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"✓ API key found: {api_key[:8]}####")
else:
    print("❌ OPENAI_API_KEY environment variable not found")
    print("Please set your API key before continuing.")

✓ API key found: sk-proj-####


### Install Required Dependencies

Install AutoGen with OpenAI support and Rich for better output formatting:

In [2]:
! pip install --quiet -U "autogen-ext[openai]>=0.7" rich

## Basic Example - simple Question-Answer

In [3]:
from rich import print as rprint
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import UserMessage

# OpenAIChatCompletionClient can retreive the api key from the OPENAI_API_KEY environment variable
openai_client = OpenAIChatCompletionClient(
    model="gpt-4o-2024-08-06"
)

result = await openai_client.create([UserMessage(content="What is the capital of France?", source="user")])
rprint(result)

# Close the client when done.
await openai_client.close()

CreateResult(
    finish_reason='stop',
    content='The capital of France is Paris.',
    usage=RequestUsage(prompt_tokens=15, completion_tokens=7),
    cached=False,
    logprobs=None,
    thought=None
)

We'll focus on a few of the `CreateResult` attributes here.

The `finish_reason` attribute contains the reason the model stopped generating text:

- `stop` - Natural completion (good!)
- `length` - Hit token limit (response may be cut off)
- `content_filter` - Blocked by safety filters
- `tool_calls` - Wants to use a function/tool
- `unknown` - Unexpected termination (API error or interruption)

The `content` attribute contains the actual AI response text - this is what you typically use.

The `usage` attribute: Token consumption for billing and monitoring:

- `prompt_tokens=15` - Your input tokens
- `completion_tokens=7` - AI's response tokens
- Total cost = based on both values

**TIP:** It is highly recommended that you take a minute to explore the `CreateResult` [api docs](https://microsoft.github.io/autogen/stable/reference/python/autogen_core.models.html#autogen_core.models.CreateResult).

## Advanced Configuration

### Controlling Response Parameters

We can control the response by providing parameters to the `create()` call.  

Here we use `json_output=True` [api docs](https://microsoft.github.io/autogen/stable/reference/python/autogen_core.models.html#autogen_core.models.ChatCompletionClient.create) to state that the response should be in json.

In this example we haven't asked OpenAI to return Json so an exception is returned.

In [4]:
from rich import print as rprint
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import UserMessage, SystemMessage

# OpenAIChatCompletionClient can retreive the api key from the OPENAI_API_KEY environment variable
openai_client = OpenAIChatCompletionClient(
    model="gpt-4o-2024-08-06"
)

try:
    result = await openai_client.create(
        [UserMessage(content="What is the capital of France?", source="user")],
        json_output=True
    )
    rprint(result)
except Exception as e:
    print(f"Received response:")
    rprint(result.content)
    print(f"Error:")
    rprint(e)

# Close the client when done.
await openai_client.close()

Received response:


The capital of France is Paris.

Error:


Error code: 400 - {'error': {'message': "'messages' must contain the word 'json' in some form, to use 
'response_format' of type 'json_object'.", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}

**NOTE**: In a later tutorial we will show you how to use a Pydantic model to provide robut data format quality checks for the LLM's output.

## Error Handling and Best Practices

In this example we should how error handling could be implemented to make your client interaction more robust.

In [5]:
import asyncio
from openai import RateLimitError, APIConnectionError, AuthenticationError

async def robust_api_call(messages, max_retries=3):
    """Make an API call with proper error handling and retries."""
    
    for attempt in range(max_retries):
        try:
            client = OpenAIChatCompletionClient(
                model="gpt-4o-2024-08-06"
            )
                
            result = await client.create(messages)
            return result
                
        except AuthenticationError as e:
            print(f"❌ Authentication error: {e}")
            print("Please check your API key.")
            break  # Don't retry auth errors
            
        except RateLimitError as e:
            wait_time = 2 ** attempt  # Exponential backoff
            print(f"⏱️ Rate limit hit. Waiting {wait_time} seconds... (attempt {attempt + 1}/{max_retries})")
            await asyncio.sleep(wait_time)
            
        except APIConnectionError as e:
            print(f"🌐 Connection error: {e}")
            if attempt < max_retries - 1:
                await asyncio.sleep(1)
                
        except Exception as e:
            print(f"❌ Unexpected error: {type(e).__name__}: {e}")
            break
    
    return None

# Example usage
async def error_handling_example():
    messages = [UserMessage(content="Hello, how are you?", source="user")]
    
    result = await robust_api_call(messages)
    
    if result:
        print(f"✅ Success: {result.content}")
    else:
        print("❌ Failed to get response after retries")

await error_handling_example()

✅ Success: Hello! I'm here and ready to help. How can I assist you today?


## Troubleshooting Common Issues

### Issue: "Authentication Error"
- **Cause**: Invalid or missing API key
- **Solution**: Verify your `OPENAI_API_KEY` environment variable

### Issue: "Rate limit exceeded"
- **Cause**: Too many requests in a short time
- **Solution**: Implement exponential backoff (see error handling example above)

### Issue: "Model not found"
- **Cause**: Invalid model name or no access to specified model
- **Solution**: Check available models in your OpenAI account

### Issue: High token usage
- **Cause**: Long conversations or verbose responses
- **Solution**: Use `max_tokens` parameter and manage conversation history length

## Next Steps: Moving to AutoGen AutoChat Framework

Now that you understand direct API calls, you're ready to explore AutoGen's more powerful features:

- **Assistants**: Pre-configured agents with specific roles and capabilities
- **Message routing**: Intelligent conversation management between multiple agents
- **Tool integration**: Agents that can call functions and use external tools
- **Conversation patterns**: Group chats, sequential conversations, and complex workflows

The next tutorial will show you how to create assistants that can collaborate, use tools, and handle complex multi-step tasks that go far beyond simple API calls.

**Key takeaway**: Use direct `autogen-core` API calls when you need simple, controlled interactions. Use the AutoGen `AgentChat` framework when you need sophisticated agent behaviors and multi-agent coordination.

## Practice Exercises

1. Try connecting to other [AI providers](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/models.html) using AutoGen's unified interface, for example:

- **Anthropic Claude**
- **Google Gemini**
- **Azure OpenAI**

2. Experiment with different models (e.g. gpt-4o-mini) and compare their responses to the same prompts!

3. Run the streaming example in the OpenAIChatCompletionClient [api docs](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.models.openai.html#autogen_ext.models.openai.OpenAIChatCompletionClient)

3. Explore the api doc links in this tutorial.  It is very important to become familiar with the api docs.